In [1]:
import  numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Load the data

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

In [3]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\DEEKSHA\\tensorflow_datasets\\mnist\\3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",

In [4]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [5]:
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

Scale the data into float 

In [6]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

In [7]:
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

Batch the data

In [8]:
BUFFER_SIZE = 10000

Shuffle the data

In [9]:
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

Extract train and validation data

In [10]:
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [11]:
print(type(train_data))

<class 'tensorflow.python.data.ops.dataset_ops.SkipDataset'>


Use mini batch gradient descent to train the model

In [12]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

#No batching needed in validation and test data because of forward propagation only
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

In [13]:
# To make the validation data have same shape and properties as train data and test data
validation_inputs, validation_targets = next(iter(validation_data))

## Outline the model

In [14]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape = (28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'),
                             tf.keras.layers.Dense(output_size, activation = 'softmax')
                            ])


## Choose Optimizer and the loss function

In [15]:
custom_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer= custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [16]:
NUM_EPOCHS = 5
model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 6s - loss: 0.2482 - accuracy: 0.9268 - val_loss: 0.1062 - val_accuracy: 0.9662 - 6s/epoch - 12ms/step
Epoch 2/5
540/540 - 4s - loss: 0.0943 - accuracy: 0.9712 - val_loss: 0.0669 - val_accuracy: 0.9802 - 4s/epoch - 7ms/step
Epoch 3/5
540/540 - 3s - loss: 0.0660 - accuracy: 0.9790 - val_loss: 0.0536 - val_accuracy: 0.9840 - 3s/epoch - 6ms/step
Epoch 4/5
540/540 - 4s - loss: 0.0477 - accuracy: 0.9845 - val_loss: 0.0478 - val_accuracy: 0.9855 - 4s/epoch - 7ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0374 - accuracy: 0.9880 - val_loss: 0.0417 - val_accuracy: 0.9882 - 4s/epoch - 7ms/step


## Test the model

In [17]:
test_loss, test_accuracy = model.evaluate(test_data)

print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 1s 541ms/step - loss: 0.0763 - accuracy: 0.9774
Test loss: 0.08. Test accuracy: 97.74%


In [22]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.BatchDataset